### Load libraries and read in data. This particular dataset has 9.5 million rows and 23 columns

In [340]:
import pandas as pd
import numpy as np
import sys

data = pd.read_csv('/Users/d/Documents/data.csv')
print(data.shape)
data.head()

(9495235, 23)


,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,...,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,...,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0
2,2355130,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:33:00,2014-06-30 23:35:00,...,327,Sheffield Ave & Webster Ave,41.921687,-87.653714,19.0,225,Halsted St & Dickens Ave,41.919936,-87.648830,15.0
3,2355129,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:26:00,2014-07-01 00:24:00,...,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,194,State St & Wacker Dr,41.887155,-87.627750,11.0
4,2355128,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:16:00,2014-06-30 23:26:00,...,320,Loomis St & Lexington St,41.872187,-87.661501,15.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0


### Check which types are assigned to which column and how much memory they use

In [341]:
memory_usage = data.memory_usage(deep=True)

overview = pd.DataFrame({'mem_usg_MB': round(memory_usage/1024/1024, 2),
                         'data_types': data.dtypes})

print('Total memory use: {} MB'.format(overview['mem_usg_MB'].sum()))
overview[1:]

Total memory use: 5733.63 MB


,mem_usg_MB,data_types
day,72.44,int64
dpcapacity_end,72.44,float64
dpcapacity_start,72.44,float64
events,572.79,object
from_station_id,72.44,int64
from_station_name,730.88,object
gender,556.91,object
hour,72.44,int64
latitude_end,72.44,float64
latitude_start,72.44,float64


### A function to check the original memory footprint of different types and the memory usage after downscaling

In [342]:
def get_memory_reduction(column, as_type):
    
    types = ['int', 'float', 'category']
    
    # Get original memory usage in MB's
    original = data[column].memory_usage(deep=True)/1024/1024
    
    # If any of the types being downsized is int/float/category, get memory usage after reduction and change column type
    if any(i in as_type for i in types):
        after_reduction = data[column].astype(as_type).memory_usage(deep=True)/1024/1024
        data[column] = data[column].astype(as_type)
    
    # if type is changed to datetime64, execute following:
    elif as_type == 'datetime':
        after_reduction = pd.to_datetime(data[column]).memory_usage(deep=True)/1024/1024
        data[column] = pd.to_datetime(data[column])
    
    print("Original memory usage of the column '{}': {} MB".format(column, round(original, 2)))
    print("Memory after downsizing to '{}': {} MB".format(as_type, round(after_reduction, 2)))
    print('Memory reduction of {}%'.format(round((1 - (after_reduction/original)) * 100, 2)))
    print()

### Numeric columns that can be downsized.

#### When checking the range of certain columns, it becomes clear that a few of them can be downsized. Features like day/week/month will always be in a specific range.



In [343]:
int8 = ['month', 'week', 'day', 'hour', 'dpcapacity_end', 'dpcapacity_start']
int16 = ['year', 'to_station_id', 'from_station_id']

for column in int8:
    get_memory_reduction(column, 'int8')

for column in int16:
    get_memory_reduction(column, 'int16')
    
get_memory_reduction('trip_id', 'int32')

Original memory usage of the column 'month': 72.44 MB
Memory after downsizing to 'int8': 9.06 MB
Memory reduction of 87.5%

Original memory usage of the column 'week': 72.44 MB
Memory after downsizing to 'int8': 9.06 MB
Memory reduction of 87.5%

Original memory usage of the column 'day': 72.44 MB
Memory after downsizing to 'int8': 9.06 MB
Memory reduction of 87.5%

Original memory usage of the column 'hour': 72.44 MB
Memory after downsizing to 'int8': 9.06 MB
Memory reduction of 87.5%

Original memory usage of the column 'dpcapacity_end': 72.44 MB
Memory after downsizing to 'int8': 9.06 MB
Memory reduction of 87.5%

Original memory usage of the column 'dpcapacity_start': 72.44 MB
Memory after downsizing to 'int8': 9.06 MB
Memory reduction of 87.5%

Original memory usage of the column 'year': 72.44 MB
Memory after downsizing to 'int16': 18.11 MB
Memory reduction of 75.0%

Original memory usage of the column 'to_station_id': 72.44 MB
Memory after downsizing to 'int16': 18.11 MB
Memory r

### DateTime. 

#### This dataframe contains 2 columns that contain dates and/or time which are saved as an object type. Converting this to DateTime64 will save memory and as a bonus, time series analysis will be possible.

In [344]:
datetime = ['starttime', 'stoptime']

for column in datetime:
    get_memory_reduction(column, 'datetime')

Original memory usage of the column 'starttime': 688.21 MB
Memory after downsizing to 'datetime': 72.44 MB
Memory reduction of 89.47%

Original memory usage of the column 'stoptime': 688.21 MB
Memory after downsizing to 'datetime': 72.44 MB
Memory reduction of 89.47%



### Changing object types to categoricals

In [345]:
category_types = ['events', 'gender', 'usertype', 'from_station_name', 'to_station_name']

for column in category_types:
    get_memory_reduction(column, 'category')
    

Original memory usage of the column 'events': 572.79 MB
Memory after downsizing to 'category': 9.06 MB
Memory reduction of 98.42%

Original memory usage of the column 'gender': 556.91 MB
Memory after downsizing to 'category': 9.06 MB
Memory reduction of 98.37%

Original memory usage of the column 'usertype': 606.71 MB
Memory after downsizing to 'category': 9.06 MB
Memory reduction of 98.51%

Original memory usage of the column 'from_station_name': 730.88 MB
Memory after downsizing to 'category': 18.18 MB
Memory reduction of 97.51%

Original memory usage of the column 'to_station_name': 730.88 MB
Memory after downsizing to 'category': 18.18 MB
Memory reduction of 97.51%



## Sparse matrices and final results.